#### Genome IDs

For every genome, download the GenBank assembly.  Make a blast database. Run tblastn for query genomes against 

+ bambooshark ASM401019v1
+ homo GRCh38.p13
+ mus GRCm38.p6
+ gallus GRCg6a
+ xenopus Xenopus_tropicalis_v9.1
+ lepisosteus LepOcu1
+ bonytongue fSclFor1.1
+ ictalurus IpCoco_1.2
+ danio GRCz11
+ esox Eluc_v4
+ gadus gadMor3.0
+ takifugu fTakRub1.2


#### Approach

Construct complete dataset that has:
 
 + query_species (homo/mus)
 + query_name (homo or mus gene that gave the hit)
 + query_start (position that query match started at)
 + query_end (position that query match ended at)
 + subject_species (genome that was searched)
 + block (chromosome/assembly/contig it is on, genbank numbering)
 + subject_start (position that subject started at)
 + subject_end (position that subject ended at)
 + e_value (hit e-value)
 + sequence (hit sequence)
 
 Save this to `all-hits.csv`
  

In [ ]:
# Set genomes to query
subject_species = ["bambooshark",
                   "homo",
                   "gallus",
                   "xenopus",
                   "lepisosteus",
                   "bonytongue",
                   "ictalurus",
                   "danio",
                   "esox",
                   "gadus",
                   "takifugu"]


In [ ]:
# Import required modules

import phylopandas as phy
from phylogenetics import tools

%matplotlib inline 
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from shapely.geometry.polygon import LinearRing, Polygon
from descartes import PolygonPatch

import re, pickle, glob, os, random, string

In [ ]:
# ---------------------------------------------------------------
# Functions for BLASTING human and zebrafish gene lists against
# genomes of various critters.
# ---------------------------------------------------------------

def location_from_description(description):
    """
    Get the genome location by parsing the description line from an ncbi
    genome fasta file sequence header.
    """
        
    try:
        location = description.split("chromosome:")[1].split(" gene:")[0].split(":")
    except IndexError:
        location = description.split("scaffold:")[1].split(" gene:")[0].split(":")
        
    for i in range(2,5):
        location[i] = int(location[i])
    
    try:
        gene_name = description.split("gene_symbol:")[1].split("description:")[0]
    except IndexError:
        gene_name = description.split("gene:")[1].split(" ")[0]

    gene_name = gene_name.strip()
    
    location.append(gene_name)
    
    return location

def load_genome(fasta_file):
    """
    Load a genome from an ncbi fasta file, extracting useful information
    into columns of the data frame.
    """

    df = phy.read_fasta(fasta_file)
    
    locations = [location_from_description(d) for d in df.description]

    df["chromosome"] = [l[1] for l in locations]
    df["start"] = [l[2] for l in locations]
    df["end"] = [l[3] for l in locations]
    df["direction"] = [l[4] for l in locations]
    df["gene_name"] = [l[5] for l in locations]

    return df
    
def range_mask(row,min_search,max_search,chromosome):
    """
    See if some part of a gene falls between min_search and max_search
    given its start and stop positions.
    """
    
    if row['chromosome'] != str(chromosome):
        return False
    
    span = [row['start'],row['end']]
    abs_start = np.min(span)
    abs_end = np.max(span)
        
    if (abs_start < min_search and abs_end >= min_search) or \
       (abs_start < max_search and abs_end >= max_search) or \
       (abs_start >= min_search and abs_end <= max_search):
        
        return True

    return False

        
def create_region_list(df,pattern,chromosome,extend_out=3000000,take_only_longest=True):
    """
    Given a genome `df`, find all genes within `extend_out` of the gene matched by
    `pattern` regular expression and on the specified `chromosome`.  
    For genes in the region with duplicate names, optionally take only the longest
    gene.
    """
    
    # Get genes that match the pattern on the appropriate chromosome
    df_pattern = df[np.logical_and(df.description.str.contains(pattern),
                                   df.chromosome==str(chromosome))]
        
    # Find the minimum and maximum extent of the gene
    all_locations = list(df_pattern.start)
    all_locations.extend(df_pattern.end)
    
    min_value = np.min(all_locations)
    max_value = np.max(all_locations)

    # Define region to search around the matched region
    min_search = min_value - extend_out
    max_search = max_value + extend_out
    
    # Find genes that overlap the range defined by min_search and max_search
    df_near = df[df.apply(range_mask,axis=1,args=(min_search,max_search,chromosome))]
    
    # Make a dictionary of all genes with identical names
    unique_set = {}
    for i in range(len(df_near.gene_name)):
        try:
            unique_set[df_near.gene_name.iloc[i]][0].append(df_near.start.iloc[i])
            unique_set[df_near.gene_name.iloc[i]][1].append(df_near.end.iloc[i])
            unique_set[df_near.gene_name.iloc[i]][2].append(i)
            unique_set[df_near.gene_name.iloc[i]][3].append(df_near.sequence.iloc[i])
        except KeyError:
            unique_set[df_near.gene_name.iloc[i]] = [[df_near.start.iloc[i]],
                                                     [df_near.end.iloc[i]],
                                                     [i],
                                                     [df_near.sequence.iloc[i]]]
    
    # Collapse genes with same name, taking:
    #     1) First entry for accession etc.
    #     2) Total extent min -> max of set of genes
    #     3) Longest protein sequence
    output = np.zeros(len(df_near.direction),dtype=np.bool)
    for g in unique_set.keys():
        
        index = unique_set[g][2][-1]
        direction = df_near.direction.iloc[index]
        
        seq_lengths = [(len(s),s) for s in unique_set[g][3]]
        seq_lengths.sort()
        sequence = seq_lengths[-1][1]
        
        df_near.iloc[index].sequence = sequence
        
        if direction > 0:
            df_near.iloc[index].start = np.min(unique_set[g][0])
            df_near.iloc[index].end = np.max(unique_set[g][1])
        if direction < 0:
            df_near.iloc[index].end = np.min(unique_set[g][1])
            df_near.iloc[index].start = np.max(unique_set[g][0])
        
        output[index] = True
    
    df_near = df_near[output]

    return df_near
    

def scaffold_blast(region_df,db):
    """
    Blast a region with a collection of genes against a blast db,
    only returning hits that match scaffold name.
    """

    all_hits = []
    for i in range(len(region_df.sequence)):
        print(i,len(region_df.sequence))

        r = tools.blast.local_blast(region_df.iloc[i],db=db,blast_program="tblastn")
        all_hits.append(r)
        
    return all_hits


def load_block_lengths(species):
    """
    Load the lengths of the blocks (contigs, chromosomes, etc.) that
    are being queried.
    """
    
    block_lengths = {}
    
    report_files = glob.glob(os.path.join(species,"*assembly_report.txt"))
    for report in report_files:
        with open(report) as f:
            for line in f.readlines():
                if not line.startswith("#"):
                    fields = line.split("\t")
                    
                    genbank_id = fields[4]
                    try:
                        seq_length = int(fields[8])
                    except ValueError:
                        seq_length = np.nan
                    
                    block_lengths[genbank_id] = seq_length
                    
    return block_lengths

# -------------------------------------------------------------------
# Functions for processing BLAST hits.
# -------------------------------------------------------------------

def merge_overlapping_genes(df_to_sort):
    """
    Merge overlapping genes. 
    """
    
    # Nothing to do
    if len(df_to_sort) < 2:
        return df_to_sort
    
    # Copy df and make a column that will encode the original order
    # of the rows
    some_df = df_to_sort.copy()
    some_df["original_order"] = np.arange(len(some_df))
    
    # Decide whether the intervals go left to right (not inverted)
    # or right to left (inverted).  Encode the smaller values as "a"
    # and the larger values as "b"
    if some_df.iloc[0].subject_start < some_df.iloc[0].subject_end:
        inverted = False
        some_df["a"] = some_df.subject_start
        some_df["b"] = some_df.subject_end
    else:
        inverted = True
        some_df["b"] = some_df.subject_start
        some_df["a"] = some_df.subject_end
    
    # Sort by "a" (the interval starts)
    some_df = some_df.sort_values(by=["a"])
    
    # Mask that encodes whether we will keep row or not
    to_keep_mask = np.ones(len(some_df),dtype=np.bool)
    
    # Go through all rows...
    list_of_ends = []
    current_end = some_df.iloc[0].b
    for i in range(1,len(some_df)):
        
        # Does new start overlap previous end? If so, merge it.
        if some_df.iloc[i].a < some_df.iloc[i-1].b:
            to_keep_mask[i] = False
            
            # Does end of the new overlap stick past current end?
            if some_df.iloc[i].b > current_end:
                current_end = some_df.iloc[i].b
        else:
            list_of_ends.append(current_end)
            current_end = some_df.iloc[i].b
    
    # List will have all ends we've seen
    list_of_ends.append(current_end)
    
    # Whack out duplicates
    some_df = some_df[to_keep_mask]
    
    # Assign new ends to intervals
    some_df.b = list_of_ends
    
    # Assign new "a" and "b" values back to start/end values
    if inverted:
        some_df.subject_start = some_df.b
        some_df.subject_end = some_df.a
    else:
        some_df.subject_start = some_df.a
        some_df.subject_end = some_df.b
    
    # Sort by original order
    some_df = some_df.sort_values(["original_order"])
    
    # Drop temporary columns and return
    return some_df.drop(["a","b","original_order"],axis=1)
    
                
    
def merge_blast_hits(subject_species_list,
                     query_hit_list,
                     query_region_list,
                     query_species_list,
                     e_value_cutoff=0.001,
                     merge_pattern="TLR4"):

    all_hits = []
    for i in range(len(query_region_list)):
        
        query_region = query_region_list[i]
        query_species = query_species_list[i]
        query_hits = query_hit_list[i]
        
        for j in range(len(subject_species_list)):
            for k in range(len(query_region.gene_name)):

                hit_df = query_hits[j][k].copy()

                # Filter by e-value
                hit_df = hit_df[hit_df.e_value < e_value_cutoff]
                if len(hit_df) == 0:
                    continue

                hit_df["query_species"] = query_species
                hit_df["query_name"] = query_region.gene_name.iloc[k]
                hit_df["subject_species"] = subject_species_list[j]
                hit_df["block"] = [h.split()[0] for h in hit_df.hit_def]

                all_hits.append(hit_df)

    all_hits = pd.concat(all_hits,ignore_index=True)
    all_hits = all_hits.reset_index(drop=True)
    
    # Look for hits where the query gene name matches the specified pattern
    compiled_pattern = re.compile(merge_pattern,flags=re.IGNORECASE)
    pattern_mask = np.array([compiled_pattern.search(q)
                             for q in all_hits.query_name],dtype=np.bool)
    pattern_hits = all_hits[pattern_mask]
    
    # Go through unique subject-species/dna block pairs in the pattern hits
    # and genes that overlap. 
    final_hits = []
    pattern_hits["unique"] = ["{}-{}".format(*pair)
                              for pair in zip(pattern_hits.subject_species,pattern_hits.block)]
    for k in set(pattern_hits.unique):
        if len(pattern_hits[pattern_hits.unique == k]) > 1:
            tmp = merge_overlapping_genes(pattern_hits[pattern_hits.unique == k])
            tmp = tmp.drop(["unique"],axis=1)
            tmp["query_name"] = merge_pattern
            final_hits.append(tmp)
    
    # Grab hits from the original data frame that do not match the pattern
    final_hits.append(all_hits[np.logical_not(pattern_mask)])
    
    # Reconstruct the total hit df
    all_hits = pd.concat(final_hits,ignore_index=True)
    all_hits = all_hits.sort_values(by=["query_species","subject_species","block","subject_start"])
    all_hits = all_hits.reset_index(drop=True)
    
    all_hits["uid"] = ["".join([random.choice(string.ascii_letters) for j in range(10)])
                                for i in range(len(all_hits))]
    
    return all_hits


In [ ]:

# Create lists of genes in the region around TLR4 for human and zebrafish
# Extend out so we have 22 genes in each dataset. 
hs = load_genome("reference/Homo_sapiens.GRCh38.pep.all.fa")
hs_region = create_region_list(hs,"TLR4|toll.like.receptor.4|toll-like receptor 4",chromosome=9,extend_out=3200000,take_only_longest=False)

dr = load_genome("reference/Danio_rerio.GRCz11.pep.all.fa")
dr_region = create_region_list(dr,"TLR4|toll.like.receptor.4|toll-like receptor 4",chromosome=13,extend_out=500000)

hs_region.to_csv("hs_region.csv")
dr_region.to_csv("dr_region.csv")


In [ ]:
# Blast genes in reference regions against genomes from different
# critters. This is very slow. It saves out all_hs_hits.pickle and
# all_dr_hits.pickle. These can be loaded in the next cell, meaning
# you only need to run this cell once, even if you tweak the 
# downstream analysis. 

hs_hits = []
dr_hits = []
for s in subject_species:
    
    block_lengths = load_block_lengths(s)
    pickle.dump(block_lengths,
                open("{}_length.pickle".format(s),"wb"))
    
    db = "{}/{}".format(s,s)
    
    print("human against {}".format(s))
    hs_hits.append(scaffold_blast(hs_region,db))
    
    print("zebrafish against {}".format(s))
    dr_hits.append(scaffold_blast(dr_region,db))

pickle.dump(hs_hits,open("all_hs_hits.pickle","wb"))
pickle.dump(dr_hits,open("all_dr_hits.pickle","wb"))


In [ ]:
# Merge all unique blast hits into a single .csv file
hs_hits = pickle.load(open("all_hs_hits.pickle","rb"))
dr_hits = pickle.load(open("all_dr_hits.pickle","rb"))

all_hits = merge_blast_hits(subject_species,
                 [hs_hits,dr_hits],
                 [hs_region,dr_region],
                 ["homo","danio"])

all_hits.to_csv("all-hits.csv")